In [1]:
from argparse import Namespace
import mlflow

import flash
from flash.text import TextClassificationData,TextClassifier

/home/lashi/miniconda3/envs/mlflow/lib/python3.8/site-packages/torchvision/models/_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and may be removed in the future. Please access them via the appropriate Weights Enum instead.
  warnings.warn(


In [2]:
args = Namespace(
    experiment_name = "/Users/lashi0812@gmail.com/mlflow",
)

In [3]:
mlflow.set_tracking_uri("databricks")

In [4]:
mlflow.set_experiment(args.experiment_name)

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/913080969227948', creation_time=1679488372636, experiment_id='913080969227948', last_update_time=1679497137089, lifecycle_stage='active', name='/Users/lashi0812@gmail.com/mlflow', tags={'mlflow.experiment.sourceName': '/Users/lashi0812@gmail.com/mlflow',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'lashi0812@gmail.com',
 'mlflow.ownerId': '3844792839736402'}>

In [5]:
experiment = mlflow.get_experiment_by_name(args.experiment_name)
experiment.experiment_id

'913080969227948'

In [6]:
mlflow.pytorch.autolog()

2023/03/22 20:29:19 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.


In [7]:
datamodule = TextClassificationData.from_csv(
    input_field="review",
    target_fields="sentiment",
    train_file="../data/imdb/train.csv",
    val_file="../data/imdb/valid.csv",
    test_file="../data/imdb/test.csv",
    batch_size=64,
)

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

/home/lashi/miniconda3/envs/mlflow/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  if await self.run_code(code, result, async_=asy):


In [8]:
model = TextClassifier(backbone="prajjwal1/bert-tiny",
                       num_classes=datamodule.num_classes)

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification

In [9]:
trainer = flash.Trainer(max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [10]:
with mlflow.start_run(experiment_id=experiment.experiment_id,
                      run_name="mlflow_run_1"):
    trainer.finetune(model,datamodule,strategy="freeze")
    trainer.test()


  | Name          | Type               | Params
-----------------------------------------------------
0 | train_metrics | ModuleDict         | 0     
1 | val_metrics   | ModuleDict         | 0     
2 | test_metrics  | ModuleDict         | 0     
3 | adapter       | HuggingFaceAdapter | 4.4 M 
-----------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

2023/03/22 20:31:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpzad98vpt/model/data, flavor: pytorch), fall back to return ['torch==1.13.1', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6287999749183655, 'test_cross_entropy': 0.6467375159263611}
--------------------------------------------------------------------------------
